In [ ]:
!pip install networkx --upgrade

In [ ]:
from IPython.core.display import HTML
from datascience import *

import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
plt.style.use('fivethirtyeight')

import networkx as nx

In [ ]:
def css_styling():
    styles = open('../../notebook_styles.css', 'r').read()
    return HTML(styles)
css_styling()

In [ ]:
# FOR DEVELOPMENT
# SEE 
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [ ]:
from client.api.notebook import Notebook 
hwk_friendship = Notebook('hw06.ok')
_ = hwk_friendship.auth(inline=True)

# Homework 06

## Part 1: Coordination cascades

Consider the graph generated by the following code snippet.

In [ ]:
g1=nx.connected_caveman_graph(3,4)
color_map={}
for node in g1:
    if node in [4,3,11]:
        color_map[node]='g'
        g1.node[node]['color']='g' # green nodes are the ones who have adopted
    else:
        color_map[node]='r'
        g1.node[node]['color']='r' # red nodes are the ones who haven't adopted yet

nx.draw(g1, with_labels=True,node_color = list(color_map.values()))

Suppose that initially the nodes with ids 4, 3 and 11 have adopted a product. Specify what other nodes will adopt the product at each iteration if the adoption threshold is 0.3. Stop when the number of nodes adopting the product is not changing.

First you need to complete the missing parts (...) in the following function definition. 

In [ ]:
#Solution
def cascade(g1):
    for node in g1:
        adopters_list=[]
        node_neighbors=[x for x in g1.neighbors(node)]

        node_attribs=nx.get_node_attributes(g1,'color')
        #print (node, node_neighbors, node_attribs)
        for each in node_attribs:
            if each in node_neighbors:
                if node_attribs[each]=='g':
                    adopters_list.append(each)
        b,a = len(adopters_list), len(node_neighbors)
        print(node, b/float(a))
        if b/float(a)>=0.3:
            g1.node[node]['color']='g'
            color_map[node]='g'
    return g1

Now run your function as many times as necessary and draw the graph each time.

In [ ]:
#Solution
g1=cascade(g1)
nx.draw(g1, with_labels=True, node_color = list(color_map.values()))

In [ ]:
#Solution
g1=cascade(g1)
nx.draw(g1, with_labels=True, node_color = list(color_map.values()))

## Part 2: The friendship paradox: <BR> Why your friends (probably) have more friends than you do

Please read this short article:

[Friends you can count on](https://opinionator.blogs.nytimes.com/2012/09/17/friends-you-can-count-on/) by Steve Strogatz  

And answer a couple of short questions about it.

If you are really curious, you can also optionally look at the original paper that discussed the friendship paradox (and which was the inspiration for this homework's title):

[Why your friends have more friends than you do](http://www.journals.uchicago.edu/doi/abs/10.1086/229693) by Scott Feld [OPTIONAL BACKGROUND]

**Question 2.1** According to Strogatz, why do people experience airplanes, restaurants, parks, and beaches to be more crowded than averages would suggest?  
*[Please answer in one or two sentences]*

<div class='response'>
[Answer here]
</div>

<div class='solution'>
Full credit:  
Because more people experience these things when they are crowded. So if an airplane flies 2 days, and on one day it carries 100 passengers and on the second day it carries 10 passengers, then 100 people experience the airplane as crowded and 10 people experience the airplane as not crowded.
</div>

**Question 2.2** Which two groups did Christakis and Fowler monitor to see who got the flu first? Which group ended up actually getting the flu first?  
*[Please answer in one or two sentences]*

<div class='response'>
[Answer here]
</div>

<div class='solution'>
Full credit:  
- the two groups were (i) randomly picked students; and (ii) friends of the randomly picked students  
- the friends of the randomly picked students got the flu first.
</div>

## Testing the friendship paradox

The friendship paradox is a famous result from social networks. In this homework, we're going to see if the friendship paradox seems to hold for the networks from the Add Health study. We'll start by loading the Add Health networks into memory, like we did in lab.

### Read in Add Health networks

In [ ]:
def read_add_health_network(network_id):
    """
    network_id : integer from 1 to 84
    
    read in the Add Health network corresponding to the given id number and
    return it as an undirected networkx object
    """

    # this file was downloaded from
    # http://moreno.ss.uci.edu/data.html#adhealth
    edge_file = os.path.join("../..", "data", "add-health", "comm" + str(network_id) + ".dat")
    with open(edge_file, 'r') as f:
        edge_lines = f.readlines()
        
    network = nx.parse_edgelist(edge_lines, nodetype=int, data=[('activity_level', float)])
    
    # note that we call the to_undirected method to ensure we get an undirected network
    return(network.to_undirected())

Now let's use this function to actually read in all 84 of the Add Health school networks:

In [ ]:
number_add_health_networks = 84
add_health_networks = [read_add_health_network(x) for x in range(1,number_add_health_networks+1)]

#### Starting with one network

We'll start by focusing on one specific network from the Add Health dataset. We'll develop some code using this one network. Later on, we'll generalize our results to all of the networks.

In [ ]:
one_network = add_health_networks[0]

**Question 2.3** Make a table that has two columns: one with the id of each node, and another with the degree of each node.

In [ ]:
degree_data = Table().with_columns([
    'id', ...,
    'degree', ...
])

degree_data

In [ ]:
#SOLUTION
degree_data = Table().with_columns([
    'id', one_network.nodes(),
    'degree', [y for x,y in one_network.degree(one_network.nodes())]
])

degree_data

In [ ]:
_ = hwk_friendship.grade('q_degree_table')

Now let's work on figuring out how to get the average degree of the neighbors of a single node.

In [ ]:
# list(one_network.nodes())
one_node = list(one_network.nodes())[0]
one_node

This bit of code will show `one_node` and some of the nodes around it:

* `one_node` itself has id 1
* the *neighbors* of `one_node` have ids 36, 37, and 52
* the neighbors of `one_node`'s neighbors (which help you see the degree of each of `one_node`'s neighbors)

NOTE: this code uses some features of the `networkx` library that we aren't going to talk about in this class. So you don't have to understand exactly how it works (though that would be a good challenge if you want one).

In [ ]:
nx.draw(one_network.subgraph(list(nx.single_source_shortest_path_length(one_network, one_node, cutoff=2).keys())),
        with_labels=True)

The `neighbors()` method will return the set of nodes that are adjacent to `one_node`:

In [ ]:
[y for y in one_network.neighbors(one_node)]

Also, you can get the degree of a specific node using the `degree` method:

In [ ]:
one_network.degree(one_node)

As we can see from the drawing above, one_node has degree 3.

You can use these facts to help answer the next question.

**Question 2.4** The code below should calculate the average degree of `one_node`'s neighbors. Fill in the missing parts.

In [ ]:
one_node_neighbors = one_network.neighbors(one_node)

nbr_degree_total = 0
num_nbrs = 0

for nbr in ...:
    nbr_degree_total = nbr_degree_total + ...
    num_nbrs = num_nbrs + 1

result = ... / ...

print("average degree of neighbors is ", result)

In [ ]:
#SOLUTION
one_node_neighbors = one_network.neighbors(one_node)

nbr_degree_total = 0
num_nbrs = 0

for nbr in one_node_neighbors:
    nbr_degree_total = nbr_degree_total + one_network.degree(nbr)
    num_nbrs = num_nbrs + 1

result = nbr_degree_total / num_nbrs

print("average degree of neighbors is ", result)

In [ ]:
_ = hwk_friendship.grade('q_one_node_nbr_avgdegree')

Now let's generalize the code you just wrote by turning it into a function. This will allow you to easily calculate the average degree of the neighbors of any node you want.

**Question 2.5** Fill in the code below to create a function that, given any network `g` and node `node`, will return the average degree of the node's neighbors.

In [ ]:
def get_average_degree_of_neighbors(g, node):
    """Given a network and a node, compute the average degree of the node's neighbors.
    
    Parameters
    ----------
    g : networkx Graph object
        The network that node is a member of
    node : networkx node (actually just an integer)
        The node
    
    Returns
    -------
    float
        The average degree of the neighbors of node
    
    """
        
    ## get the nodes that are the neighbors of node
    node_neighbors = ...
    
    nbr_degree_total = 0
    num_nbrs = 0
    
    ## get the degrees of each of those nodes
    for nbr in node_neighbors:
        nbr_degree_total = nbr_degree_total + ...
        num_nbrs = num_nbrs + 1
        
    ## calculate the average
    avg_nbr_degree = ... / ...
    
    ## return it
    return(avg_nbr_degree)

In [ ]:
# SOLUTION
def get_average_degree_of_neighbors(g, node):
    """Given a network and a node, compute the average degree of the node's neighbors.
    
    Parameters
    ----------
    g : networkx Graph object
        The network that node is a member of
    node : networkx node (actually just an integer)
        The node
    
    Returns
    -------
    float
        The average degree of the neighbors of node
    
    """
        
    ## get the nodes that are the neighbors of node
    node_neighbors = g.neighbors(node)
    
    nbr_degree_total = 0
    num_nbrs = 0
    
    ## get the degrees of each of those nodes
    for nbr in node_neighbors:
        nbr_degree_total = nbr_degree_total + g.degree(nbr)
        num_nbrs = num_nbrs + 1
        
    ## calculate the average
    avg_nbr_degree = nbr_degree_total / num_nbrs
    
    ## return it
    return(avg_nbr_degree)

In [ ]:
_ = hwk_friendship.grade('q_nbr_avgdegree_function')

**Question 2.6** Now use the function you wrote to calculate the average of the neighbors' degrees for every node in `one_network`.

In [ ]:
avg_friends_degree = make_array()

for node in ...:
    avg_friends_degree = np.append(avg_friends_degree, ...)

nbr_avg_degrees = Table().with_columns([
    'id', ...,
    'avg_friends_degree', ...
])

nbr_avg_degrees

In [ ]:
#SOLUTION
avg_friends_degree = make_array()

for node in one_network.nodes():
    avg_friends_degree = np.append(avg_friends_degree, 
                                   get_average_degree_of_neighbors(one_network, node))

nbr_avg_degrees = Table().with_columns([
    'id', one_network.nodes(),
    'avg_friends_degree', avg_friends_degree
])

nbr_avg_degrees

In [ ]:
_ = hwk_friendship.grade('q_avg_friends_degree')

**Question 2.7** Now you have created a table `degree_data` which has (node id, node degree) and a second table `nbr_avg_degrees` that has (node id, average of friends' degrees). Join these two tables together so that you have a Table with (node id, node degree, average of friends' degrees).

In [ ]:
friend_data = ....join('id', ...)
friend_data

In [ ]:
#SOLUTION
friend_data = degree_data.join('id', nbr_avg_degrees)
friend_data

In [ ]:
_ = hwk_friendship.grade('q_friend_data')

**Question 2.8** What does the friendship paradox predict about the values in the 'degree' and 'avg_friends_degree' column of the `friend_data` Table that you just made? Does it say that (i) on average, they should be about the same; (ii) on average, `degree` should be bigger than `avg_friends_degree`; or (iii) on average, `avg_friends_degree` should be bigger than `degree`?

<div class='response'>
[Answer here]
</div>

<div class='solution'>
Full credit:  
- (iii) on average, `avg_friends_degree` should be bigger than `degree`
</div>

**Question 2.9** Now make a scatter plot that shows at the relationship between the degree of each node (x axis) and the average degree of the node's friends (y axis).

In [ ]:
...

In [ ]:
#SOLUTION
friend_data.scatter('degree', 'avg_friends_degree')

**Question 2.10** Does the plot you just made seem consistent with what would be predicted from the friendship paradox?

<div class='response'>
[Answer here]
</div>

<div class='solution'>
Full credit:  
Yes, it does: many of the points lie above the $y=x$ line, meaning that `avg_friends_degree` is bigger than `degree`.  
NOTE: you could argue that this result is unclear and receive full credit if you give a good reason (we aren't doing a formal test, so there is no proof one way or another).
</div>

#### Write a function that calculates the fraction of nodes whose degree is less than the average of their neighbors.

The plot you just made investigated the friendship paradox by looking at every single node in one network. Now we are going to try to look at all of the different networks in the Add Health study.

One way to do so would be to look at every single node across all of the networks in the Add Health study. However, we're going to try a different approach: we're going to develop a metric that can be calculated once for every network, and then we'll compare that metric across the different networks in the Add Health study.

The metric we'll look at is the fraction of nodes in the network whose degree is smaller than the average of its friends' degrees. Intuitively, when this metric is high, then many nodes in the network experience the friendship paradox (because they have fewer friends than the average of their friends).

**Question 2.11** The function below should take a network and calculate the fraction of the nodes in the network that have degree smaller than the average of its friends' degrees. Fill in the missing parts. (This function should make use of the work you did above.)

In [ ]:
def frac_degree_lt_neighbors(g):
    degree = make_array()
    avg_friends_degree = make_array()

    for node in g.nodes():
        degree = np.append(degree, ...)
        avg_friends_degree = np.append(avg_friends_degree, ...)

    # calculate the fraction of nodes whose degree is smaller than the average of
    # their friends' degrees and return it
    # (HINT: you should fill in a boolean expression here)
    return(np.mean(...))

In [ ]:
#SOLUTION
def frac_degree_lt_neighbors(g):
    degree = make_array()
    avg_friends_degree = make_array()

    for node in g.nodes():
        degree = np.append(degree, g.degree(node))
        avg_friends_degree = np.append(avg_friends_degree, get_average_degree_of_neighbors(g, node))

    # calculate the fraction of nodes whose degree is smaller than the average of
    # their friends' degrees and return it
    return(np.mean(degree < avg_friends_degree))

In [ ]:
_ = hwk_friendship.grade('q_frac_degree_lt')

Here is a function that takes the function that you wrote above and uses it to calculate the average number of neighbors of each node in a given network. So this function:

1. For each node in the network, calculate the average number of friends that the node's friends have
2. Take the average of that quantity over all of the nodes in the network

We'll use this function below.

In [ ]:
def get_avg_nbr_degree(g):
    avg_nbr_degree_total = 0
    num_nodes = 0
    
    for node in g.nodes():
        avg_nbr_degree_total = avg_nbr_degree_total + get_average_degree_of_neighbors(g, node)
        num_nodes = num_nodes + 1
        
    return(avg_nbr_degree_total / num_nodes)

To see an example of the function in action, try this out:

In [ ]:
get_avg_nbr_degree(one_network)

This means that the average node in the network `one_network` has friends whose average degree is about 7.7

#### Apply the function to calculate the average degree and average friends' degree for all add health networks.

**Question 2.12** Now go through and, for each Add Health network, calculate (i) the average degree; (ii) the average of each node's neighbors' degrees; (iii) the fraction of nodes for which the degree is smaller than the average of its neighbors' degrees.

*[HINT: You might find it helpful to follow the pattern we used in lab here.]*

In [ ]:
avg_degree = make_array()
avg_neighbor_degree = make_array()
frac_smaller_than_neighbors = make_array()

for g in add_health_networks:
    avg_degree = np.append(avg_degree, ...)
    avg_neighbor_degree = np.append(avg_neighbor_degree, ...)
    frac_smaller_than_neighbors = np.append(frac_smaller_than_neighbors, ...)

add_health_msmts = Table().with_columns([
     'id', np.arange(1, number_add_health_networks+1),
     'num_nodes', num_nodes,
     'num_edges', num_edges,
     'avg_degree', avg_degree,
     'avg_neighbor_degree', avg_neighbor_degree,
     'frac_lt_neighbors', frac_smaller_than_neighbors
    ])

In [ ]:
#SOLUTION
avg_degree = make_array()
avg_neighbor_degree = make_array()
frac_smaller_than_neighbors = make_array()

for g in add_health_networks:
    avg_degree = np.append(avg_degree, 2.0*g.number_of_edges()/g.number_of_nodes())
    avg_neighbor_degree = np.append(avg_neighbor_degree, get_avg_nbr_degree(g))
    frac_smaller_than_neighbors = np.append(frac_smaller_than_neighbors, frac_degree_lt_neighbors(g))

add_health_msmts = Table().with_columns([
     'id', np.arange(1, number_add_health_networks+1),
     'avg_degree', avg_degree,
     'avg_neighbor_degree', avg_neighbor_degree,
     'frac_lt_neighbors', frac_smaller_than_neighbors
    ])

In [ ]:
str(round(np.corrcoef(add_health_msmts['avg_neighbor_degree'], add_health_msmts['frac_lt_neighbors'])[0,1], 5))

In [ ]:
_ = hwk_friendship.grade('q_add_health_msmts')

**Question 2.13** Make a scatterplot that compares the average degree (x axis) and the average neighbor degree (y axis) across all of the Add Health networks.

In [ ]:
...

In [ ]:
#SOLUTION
add_health_msmts.scatter('avg_degree', 'avg_neighbor_degree')

**Question 2.14** Does the scatterplot you just made seem to be consistent with the friendship paradox?

<div class='response'>
[Answer here]
</div>

<div class='solution'>
Full credit:  
- Yes, it does: all of the points lie above the $y=x$ line, meaning that in every single Add Health network, the average neighbor degree is larger than the average degree.
</div>

**Question 2.15** Make a histogram that shows, across all of the Add Health networks, the distribution of the fraction of nodes whose degree is smaller than the neighbors' average degree.

In [ ]:
...

In [ ]:
#SOLUTION
add_health_msmts.hist('frac_lt_neighbors')

**Question 2.16** Does the histogram you just made seem to be consistent with what you would expect from the friendship paradox?

<div class='response'>
[Answer here]
</div>

<div class='solution'>
Full credit:  
- You could argue yes, it does: in all networks, well over half of the nodes will have friends whose average degree is higher than their own.
- You could also argue that it is not clear, since we have not set up a formal test, and so we don't have a good way to tell what we would expect to see under the friendship paradox. (This is a weaker argument, in my opinion, but it isn't wrong.)
</div>

# Submit the homework


In [ ]:
_ = hwk_friendship.submit()